## Multiple Linear Regression
### and backward elimination

The purpose of this notebook is to 
* apply Multiple Linear Regression to the preprocessed dataset
* apply backward elimination to the model
* ultimately find out the independent variables (World Development Idicators) which influence the dependent variable (Happy Planet Index) the most.

The model will be applied to the "wdi_hpi_2016_df" dataset, which was created in the Data Preprocessing JNotebook. This dataset is based on
* the Happy Planet Index for 2016 (see https://happyplanetindex.org/),
* the World Development Indicators (1960 - 2019) by the World Bank (see https://datacatalog.worldbank.org/dataset/world-development-indicators)

In [1]:
# Import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import dataset
dataset = pd.read_pickle('../data/wdi_hpi_2016_df.pkl')
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, dataset.shape[1]-1].values

In [3]:
# Split datasets into Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [4]:
# Fit Multiple Linear Regression Model to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [5]:
# Predict the Test set results
y_pred = regressor.predict(X_test)

In [6]:
# Reduce less important variables with Backward Elimination
import statsmodels.regression.linear_model as sm
# for statsmodel to understand the multiple linear regression equation a new column with b0 equals one is required (y = b0 + b1*x1 + b2*x2 + ... + bn*xn)
X_opt = np.append(arr = np.ones((len(X), 1)).astype(int), values = X, axis = 1)   # Add X to the newly created array of 1s
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     4.339
Date:                Wed, 12 Feb 2020   Prob (F-statistic):           4.13e-07
Time:                        18:09:34   Log-Likelihood:                -307.15
No. Observations:                 139   AIC:                             808.3
Df Residuals:                      42   BIC:                             1093.
Df Model:                          96                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -26.2703     44.902     -0.585      0.562    -116.886      64.345
x1             0.0054      0.003      2.120      0.040       0.000       0.011
x2            -0.0196      0.040     -0.489      0.627      -0.101       0.061
x3            -0.1998      0.166     -1.206      0.235      -0.534       0.135
x4            -0.0351      0.039     -0.899      0.374      -0.114       0.044
x5            -0.0623      0.040     -1.546      0.130      -0.144       0.019
x6             0.0338      0.031      1.101      0.277      -0.028       0.096
x7            -0.0560      0.053     -1.050      0.300      -0.164       0.052
x8            -0.0196      0.069     -0.282      0.779      -0.160       0.120
x9             0.2253      0.153      1.469      0.149      -0.084       0.535
x10           -0.0057      0.076     -0.075      0.940      -0.160       0.148
x11           -0.0463      0.100     -0.463      0.646      -0.248       0.156
x12         7.738e-12   8.65e-12      0.894      0.376   -9.73e-12    2.52e-11
x13         5.451e-11   3.32e-11      1.641      0.108   -1.25e-11    1.22e-10
x14        -8.277e-12    1.7e-10     -0.049      0.961   -3.52e-10    3.35e-10
x15            0.1027      0.126      0.812      0.421      -0.152       0.358
x16            0.0389      0.050      0.783      0.438      -0.061       0.139
x17         9.053e-10   4.57e-10      1.982      0.054   -1.63e-11    1.83e-09
x18           -0.0076      0.009     -0.805      0.425      -0.027       0.011
x19           -0.1230      0.073     -1.686      0.099      -0.270       0.024
x20            0.0014      0.002      0.591      0.558      -0.003       0.006
x21            0.1179      0.101      1.166      0.250      -0.086       0.322
x22           -0.0499      0.066     -0.755      0.455      -0.184       0.084
x23           -0.1460      0.129     -1.129      0.265      -0.407       0.115
x24            0.0004      0.023      0.019      0.985      -0.045       0.046
x25            0.1036      0.331      0.312      0.756      -0.565       0.772
x26           -0.0040      0.067     -0.059      0.953      -0.138       0.130
x27           -0.0122      0.023     -0.528      0.600      -0.059       0.035
x28            0.0033      0.027      0.121      0.904      -0.052       0.058
x29           -0.1713      0.255     -0.673      0.505      -0.685       0.342
x30           -1.1039      1.333     -0.828      0.412      -3.793       1.585
x31            0.2351      0.109      2.160      0.037       0.015       0.455
x32            0.1986      0.097      2.043      0.047       0.002       0.395
x33            0.5576      0.170      3.283      0.002       0.215       0.900
x34           -1.1936      0.438     -2.728      0.009      -2.077      -0.311
x35            0.7916      0.300      2.643      0.012       0.187       1.396
x3

In [7]:
# Fuction to automatically remove columns where P-value is below significance level of 5%
'''
def backward_elimination(x, significance):
    num_vars = len(x[0])
    temp = np.zeros((X_opt.shape)).astype(int)
    for i in range(0, num_vars):
        regressor_OLS = sm.OLS(endog = y, exog = x).fit()
        max_var = max(regressor_OLS.pvalues).astype(float)
        adjR_before = regressor_OLS.rsquared_adj.astype(float)
        if max_var > significance:
            for j in range(0, num_vars - i):
                if (regressor_OLS.pvalues[j].astype(float) == max_var):
                    temp[:,j] = x[:, j]
                    x = np.delete(x, j, axis = 1)
                    tmp_regressor = sm.OLS(endog = y, exog = x).fit()
                    adjR_after = tmp_regressor.rsquared_adj.astype(float)
                    if (adjR_before >= adjR_after):
                        x_rollback = np.hstack((x, temp[:,[0,j]]))
                        x_rollback = np.delete(x_rollback, j, 1)
                        print (regressor_OLS.summary())
                        return x_rollback
                    else:
                        continue
    regressor_OLS.summary()
    return x

significance = 0.05
X_opt = X_opt[:, list(range(X_opt.shape[1]))]
X_opt2 = backward_elimination(X_opt, significance)
'''

'\ndef backward_elimination(x, significance):\n    num_vars = len(x[0])\n    temp = np.zeros((X_opt.shape)).astype(int)\n    for i in range(0, num_vars):\n        regressor_OLS = sm.OLS(endog = y, exog = x).fit()\n        max_var = max(regressor_OLS.pvalues).astype(float)\n        adjR_before = regressor_OLS.rsquared_adj.astype(float)\n        if max_var > significance:\n            for j in range(0, num_vars - i):\n                if (regressor_OLS.pvalues[j].astype(float) == max_var):\n                    temp[:,j] = x[:, j]\n                    x = np.delete(x, j, axis = 1)\n                    tmp_regressor = sm.OLS(endog = y, exog = x).fit()\n                    adjR_after = tmp_regressor.rsquared_adj.astype(float)\n                    if (adjR_before >= adjR_after):\n                        x_rollback = np.hstack((x, temp[:,[0,j]]))\n                        x_rollback = np.delete(x_rollback, j, 1)\n                        print (regressor_OLS.summary())\n                        r

In [10]:
import statsmodels.formula.api as sm
def backwardElimination(x, sl):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
    regressor_OLS.summary()
    return x
         
SL = 0.05
X_opt = X_opt[:, list(range(X_opt.shape[1]))]
X_Modeled = backwardElimination(X_opt, SL)
X_Modeled.shape

(139, 39)